In [8]:
import tensorflow as tf
import numpy as np
from numpy import matlib
def innerPd(pt0, pt1):
    return tf.mul(pt0[:,0],pt1[:,0]) + tf.mul(pt0[:,1],pt1[:,1])
def projPt(lnPt0, lnPt1, pt):
    pt = tf.sub(pt , lnPt0)
    lnPt1 = tf.sub(lnPt1 , lnPt0)
    lnLen = tf.sqrt(innerPd(lnPt1, lnPt1))
    coefs3 = innerPd(pt, lnPt1)
    coefs = tf.mul( tf.sign(coefs3) , tf.sqrt( tf.abs(coefs3) ) )
    coefs2 = tf.div(coefs, lnLen)
    dup1 = tf.pack([coefs2, coefs], axis = 1)
    ppp = tf.mul( dup1  , lnPt1 /2) 
    pjs = lnPt0 + ppp
    return pjs
def distLmtd(lnPt0, lnPt1, pt):
    pj = projPt(lnPt0, lnPt1, pt)
    
    dist1 = tf.sub(pt[0] , pj[:, 0])
    dist1 = tf.pow(dist1, 2)
    
    dist2 = tf.sub(pt[1] , pj[:, 1])
    dist2 = tf.pow(dist2, 2)
    
    dist = tf.pow(tf.add(dist1, dist2), 0.5)
    
    return (dist, pj)
def closeTo(allMat, sign):
    filter_ = tf.ones( [allMat.shape[0]])
    filter_ = tf.equal(filter_, filter_ )
    idRules = [[0,0],[0,2],[1,1],[1,3]]
    for rule in idRules:
        t1 = allMat[:, rule[1]] - sign[ rule[0] ]
        t2 = tf.less( tf.abs( t1 ), 0.004 )
        filter_ = tf.logical_and( filter_ , t2 )
    inds_ = tf.where(filter_)

    allMatTf = tf.constant(allMat)
    ged_ = tf.gather(allMatTf, inds_)

    ged_ = tf.reshape(ged_, [-1,4])
    return ged_
def findClosest(allPairsMat_, sign):
    allPairsMat_ = closeTo(allPairsMat_, sign)
    dists, pjs = distLmtd( allPairsMat_[:,0:2], allPairsMat_[:,2:4], sign )
    minIdx = tf.argmin(dists, 0)
    minDist = tf.gather(dists, minIdx)
    minPair = tf.gather(allPairsMat_, minIdx)
    minPjs = tf.gather(pjs, minIdx)
    return ( minDist, minPair, minPjs, pjs, dists, allPairsMat_ )

allPairsMat_ = np.load('allPairsMat.npy')
def calcFunc(sign):
    dist_, line_, _, _, _, _ = findClosest(allPairsMat_, sign)
    return [dist_, line_[0], line_[1], line_[2], line_[3] ]

import numpy as np
import pandas as pd

signals = pd.read_csv('signalisation.csv')
parkingSignals = signals[signals['DESCRIPTION_CAT'] == 'STATIONNEMENT']
parkingSignals = parkingSignals.iloc[0:]
parkingSignalCoordinates = np.matrix( [parkingSignals['Latitude'].values, parkingSignals['Longitude'].values] ).T

def processJobAllS(jobAllS):
    res = tf.map_fn( calcFunc , jobAllS, dtype=[tf.float64, tf.float64, tf.float64, tf.float64, tf.float64] )
    return res

in_vals = parkingSignalCoordinates

print('bulding the tensors...')

in_ = tf.placeholder(tf.float64, in_vals.shape)

num_workers = 4
batchsize = int(in_vals.shape[0] / 4)

ins_ = []
outs_ = []

for workerId in range(num_workers):
    with tf.device("/job:tfworker/task:0"):
        ins_.append( in_[ workerId*batchsize : (workerId+1)*batchsize ] )
        outs_.append( processJobAllS( ins_[workerId] ) )
        if workerId == num_workers-1:
            out_ = tf.pack(outs_, axis = 0)

import time
start_time = time.time()
print('starting processing ', str(in_vals.shape[0]), ' records.')
            
with tf.Session("grpc://localhost:2222") as sess:
    out_vals = sess.run(out_, feed_dict={in_:in_vals})

print('done with the tensorflow session.')
print('it took ' + str(time.time() - start_time) + ' seconds.')
    
finalres = np.zeros([in_vals.shape[0], 5])
batchSize = out_vals[0].shape[1]
for i in range(out_vals.shape[0]):
    finalres[batchSize*i:batchSize*(i+1), :] = np.asmatrix(out_vals[i].T)

print(finalres.shape)

parkingSignals['segmentX1'] = finalres[:,1]
parkingSignals['segmentY1'] = finalres[:,2]
parkingSignals['segmentX2'] = finalres[:,3]
parkingSignals['segmentY2'] = finalres[:,4]

parkingSignals[['Latitude', 'Longitude', 'segmentX1', 'segmentY1', 'segmentX2', 'segmentY2']].head()
parkingSignals.to_pickle('parking_signs_segments.pkl')

bulding the tensors...
starting processing  126360  records.
done with the tensorflow session.
it took 264.7268626689911 seconds.
(126360, 5)


In [10]:

import folium
%matplotlib inline
map_ = folium.Map(location=[45.5017, -73.5673], zoom_start=11)

colors = ['red', 'dardred', 'blue', 'darkblue', 'green', 'darkgreen']
for idx in range(20):
    i = np.random.randint(0, parkingSignals.shape[0])
    row = parkingSignals.iloc[i]
    signal_coor = ( row['Latitude'], row['Longitude'] )
    line = [(row['segmentX1'], row['segmentY1']), (row['segmentX2'], row['segmentY2']  )]
    c_ = colors[np.random.randint(0, len(colors))]
    folium.Marker(signal_coor, popup = 'sign' + str(i), icon = folium.Icon(color =c_)).add_to(map_)
    folium.PolyLine( line , color=c_, weight=2.5, opacity=1).add_to(map_)
#map_


<img src="./closestSegments.bmp">